In [17]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Paths to folders containing images
cancer_folder_path = r"D:\yedek\lavdie.rada\Pictures\kancer"
non_cancer_folder_path = r"D:\yedek\lavdie.rada\Pictures\non cancer"

# Image dimensions
IMG_HEIGHT = 128
IMG_WIDTH = 128

# Load images and labels
def load_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            img = tf.keras.preprocessing.image.load_img(file_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            images.append(img_array)
            labels.append(label)
        except Exception as e:
            print(f"Failed to process {file_path}: {e}")
    return np.array(images), np.array(labels)

# Load cancer and non-cancer images
cancer_images, cancer_labels = load_images(cancer_folder_path, label=1)  # 1 for cancer
non_cancer_images, non_cancer_labels = load_images(non_cancer_folder_path, label=0)  # 0 for non-cancer

# Combine datasets
X = np.concatenate((cancer_images, non_cancer_images), axis=0)
y = np.concatenate((cancer_labels, non_cancer_labels), axis=0)

# Normalize pixel values to [0, 1]
X = X / 255.0

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Plot training history
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Save the model
#model.save('cancer_classifier_model.h5')


Failed to process D:\yedek\lavdie.rada\Pictures\kancer\Cancer: [Errno 13] Permission denied: 'D:\\yedek\\lavdie.rada\\Pictures\\kancer\\Cancer'
Failed to process D:\yedek\lavdie.rada\Pictures\kancer\cancercases: [Errno 13] Permission denied: 'D:\\yedek\\lavdie.rada\\Pictures\\kancer\\cancercases'
Failed to process D:\yedek\lavdie.rada\Pictures\non cancer\1st batch: [Errno 13] Permission denied: 'D:\\yedek\\lavdie.rada\\Pictures\\non cancer\\1st batch'
Failed to process D:\yedek\lavdie.rada\Pictures\non cancer\2nd batch: [Errno 13] Permission denied: 'D:\\yedek\\lavdie.rada\\Pictures\\non cancer\\2nd batch'


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data directories
cancer_dir = r"C:\Users\kenza.chenni\Desktop\acıbademsana\kancer"
non_cancer_dir = r"C:\Users\kenza.chenni\Desktop\acıbademsana\non cancer"

# Data preprocessing
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Load datasets
cancer_dataset = datasets.ImageFolder(root=cancer_dir, transform=transform)
non_cancer_dataset = datasets.ImageFolder(root=non_cancer_dir, transform=transform)

# Combine datasets
full_dataset = torch.utils.data.ConcatDataset([cancer_dataset, non_cancer_dataset])
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Define CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, activation='relu')
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, activation='relu')
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 31 * 31, 128)
        self.fc2 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 31 * 31)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc2(x))
        return x

# Instantiate and compile
model = CNN().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.float().to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# Evaluate
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


ModuleNotFoundError: No module named 'torch'

In [19]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cpu
     ---------------------------------------- 0.0/205.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/205.4 MB ? eta -:--:--
Note: you may need to restart the kernel to use updated packages.


ERROR: Exception:
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 102, in read
    self.__buf.write(data)
  File "C:\ProgramData\anaconda3\Lib\tempfile.py", line 499, in func_wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last)

In [21]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from google.colab import drive

# Paths to folders containing images
cancer_folder_path = r"C:\Users\kenza.chenni\Desktop\acıbademsana\kancer"
non_cancer_folder_path = r"C:\Users\kenza.chenni\Desktop\acıbademsana\non cancer"


# Image dimensions
IMG_HEIGHT = 128
IMG_WIDTH = 128

# Load images using ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)

# Load datasets with ImageDataGenerator
cancer_dataset = datagen.flow_from_directory(
    cancer_folder_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='binary',  # For binary classification
    color_mode='rgb'
)

non_cancer_dataset = datagen.flow_from_directory(
    non_cancer_folder_path,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='binary',
    color_mode='rgb'
)

# Split datasets into train and validation sets
train_dataset = tf.data.Dataset.from_generator(
    lambda: zip(cancer_dataset, non_cancer_dataset),
    output_signature=(
        tf.TensorSpec(shape=(None, IMG_HEIGHT, IMG_WIDTH, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.int32)
    )
)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, epochs=10)

# Evaluate the model (you can also evaluate it using validation data from ImageDataGenerator)
loss, accuracy = model.evaluate(train_dataset)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Plot training history
plt.plot(history.history['accuracy'], label='train accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Save the model
# model.save('/content/drive/MyDrive/cancer_classifier_model.h5')


ImportError: cannot import name 'pywrap_tensorflow' from 'tensorflow.python' (C:\Users\kenza.chenni\AppData\Roaming\Python\Python312\site-packages\tensorflow\python\__init__.py)

In [5]:
!pip install tensorflow



Defaulting to user installation because normal site-packages is not writeable
  Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl.metadata (3.3 kB)
  Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl.metadata (4.9 kB)
Using cached tensorflow-2.18.0-cp312-cp312-win_amd64.whl (7.5 kB)
Using cached tensorflow_intel-2.18.0-cp312-cp312-win_amd64.whl (390.3 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [9]:
import os
from PIL import Image

# Define the folder path
folder_path = r'C:\Users\kenza.chenni\Desktop\acıbademsana'

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Loop through the files and check for image files
for file in files:
    file_path = os.path.join(folder_path, file)
    
    # Check if the file is an image (you can add more formats if needed)
    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        try:
            # Open the image
            img = Image.open(file_path)
            img.show()  # Show the image
            print(f"Image found and displayed: {file}")
        except Exception as e:
            print(f"Error opening image {file}: {e}")


In [11]:
from PIL import Image

# Define the image file path
image_path = r'C:\Users\kenza.chenni\Desktop\acıbademsana\non cancer\3rd batch\20150204\1.2.840.113681.2225186874.923.3600518121.32\1.2.840.113681.2225186874.923.3600518121.33.1'

# Open and display the image
try:
    img = Image.open(image_path)
    img.show()  # Display the image
except Exception as e:
    print(f"Error opening the image: {e}")


Error opening the image: [Errno 13] Permission denied: 'C:\\Users\\kenza.chenni\\Desktop\\acıbademsana\\non cancer\\3rd batch\\20150204\\1.2.840.113681.2225186874.923.3600518121.32\\1.2.840.113681.2225186874.923.3600518121.33.1'


In [15]:
from PIL import Image
import os

# Define the folder path
folder_path = r'D:\yedek\lavdie.rada\Pictures'

# Get a list of all files in the folder
files = os.listdir(folder_path)

# Loop through the files and check for image files
for file in files:
    file_path = os.path.join(folder_path, file)
    
    # Check if the file is an image (you can add more formats if needed)
    if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        try:
            # Open the image
            img = Image.open(file_path)
            img.show()  # Show the image
            print(f"Image found and displayed: {file}")
        except Exception as e:
            print(f"Error opening image {file}: {e}")


Image found and displayed: 1.2.840.113681.2225186874.5523.3526381414.205.1.jpg


In [19]:
import os

# Paths to folders containing images
cancer_folder_path = r"D:\yedek\lavdie.rada\Pictures\kancer"
non_cancer_folder_path = r"D:\yedek\lavdie.rada\Pictures\non cancer"

# Function to check if the folder exists and is accessible
def check_folder_access(folder_path):
    if os.path.exists(folder_path):
        if os.access(folder_path, os.R_OK):
            print(f"Access granted to: {folder_path}")
        else:
            print(f"Permission denied to access: {folder_path}")
    else:
        print(f"Folder does not exist: {folder_path}")

# Check access for both folders
check_folder_access(cancer_folder_path)
check_folder_access(non_cancer_folder_path)


Access granted to: D:\yedek\lavdie.rada\Pictures\kancer
Access granted to: D:\yedek\lavdie.rada\Pictures\non cancer


In [23]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Paths to folders containing images
cancer_folder_path = r"D:\yedek\lavdie.rada\Pictures\kancer"
non_cancer_folder_path = r"D:\yedek\lavdie.rada\Pictures\non cancer"

# Image dimensions
IMG_HEIGHT = 128
IMG_WIDTH = 128

# Load images and labels
def load_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Skip directories, only process files
        if os.path.isdir(file_path):
            print(f"Skipping directory: {filename}")
            continue
        
        # Check if the file is an image (add more formats if needed)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            try:
                img = tf.keras.preprocessing.image.load_img(file_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
                img_array = tf.keras.preprocessing.image.img_to_array(img)
                images.append(img_array)
                labels.append(label)
            except Exception as e:
                print(f"Failed to process {file_path}: {e}")
        else:
            print(f"Skipping non-image file: {filename}")
    return np.array(images), np.array(labels)

# Load cancer and non-cancer images
cancer_images, cancer_labels = load_images(cancer_folder_path, label=1)  # 1 for cancer
non_cancer_images, non_cancer_labels = load_images(non_cancer_folder_path, label=0)  # 0 for non-cancer

# Combine datasets
X = np.concatenate((cancer_images, non_cancer_images), axis=0)
y = np.concatenate((cancer_labels, non_cancer_labels), axis=0)

# Normalize pixel values to [0, 1]
X = X / 255.0

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Plot training history
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Save the model
# model.save('cancer_classifier_model.h5')


Skipping directory: Cancer
Skipping directory: cancercases
Skipping directory: 1st batch
Skipping directory: 2nd batch


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [25]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Paths to folders containing images
cancer_folder_path = r"D:\yedek\lavdie.rada\Pictures\kancer"
non_cancer_folder_path = r"D:\yedek\lavdie.rada\Pictures\non cancer"

# Image dimensions
IMG_HEIGHT = 128
IMG_WIDTH = 128

# Load images and labels
def load_images(folder_path, label):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Print out the file type for debugging
        print(f"Processing: {file_path} (is directory: {os.path.isdir(file_path)})")
        
        # Skip directories, only process files
        if os.path.isdir(file_path):
            print(f"Skipping directory: {filename}")
            continue
        
        # Check if the file is an image (add more formats if needed)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
            try:
                img = tf.keras.preprocessing.image.load_img(file_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
                img_array = tf.keras.preprocessing.image.img_to_array(img)
                images.append(img_array)
                labels.append(label)
            except Exception as e:
                print(f"Failed to process {file_path}: {e}")
        else:
            print(f"Skipping non-image file: {filename}")
    return np.array(images), np.array(labels)

# Load cancer and non-cancer images
cancer_images, cancer_labels = load_images(cancer_folder_path, label=1)  # 1 for cancer
non_cancer_images, non_cancer_labels = load_images(non_cancer_folder_path, label=0)  # 0 for non-cancer

# Combine datasets
X = np.concatenate((cancer_images, non_cancer_images), axis=0)
y = np.concatenate((cancer_labels, non_cancer_labels), axis=0)

# Normalize pixel values to [0, 1]
X = X / 255.0

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Plot training history
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Save the model
# model.save('cancer_classifier_model.h5')


Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer (is directory: True)
Skipping directory: Cancer
Processing: D:\yedek\lavdie.rada\Pictures\kancer\cancercases (is directory: True)
Skipping directory: cancercases
Processing: D:\yedek\lavdie.rada\Pictures\non cancer\1st batch (is directory: True)
Skipping directory: 1st batch
Processing: D:\yedek\lavdie.rada\Pictures\non cancer\2nd batch (is directory: True)
Skipping directory: 2nd batch


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [27]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Paths to folders containing images
cancer_folder_path = r"D:\yedek\lavdie.rada\Pictures\kancer"
non_cancer_folder_path = r"D:\yedek\lavdie.rada\Pictures\non cancer"

# Image dimensions
IMG_HEIGHT = 128
IMG_WIDTH = 128

# Load images and labels recursively
def load_images(folder_path, label):
    images = []
    labels = []
    for root, dirs, files in os.walk(folder_path):  # Using os.walk to walk through subdirectories
        for filename in files:
            file_path = os.path.join(root, filename)
            
            # Print the path and type for debugging
            print(f"Processing: {file_path} (is directory: {os.path.isdir(file_path)})")
            
            # Check if the file is an image (you can add more formats if needed)
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                try:
                    img = tf.keras.preprocessing.image.load_img(file_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
                    img_array = tf.keras.preprocessing.image.img_to_array(img)
                    images.append(img_array)
                    labels.append(label)
                except Exception as e:
                    print(f"Failed to process {file_path}: {e}")
            else:
                print(f"Skipping non-image file: {filename}")
    return np.array(images), np.array(labels)

# Load cancer and non-cancer images
cancer_images, cancer_labels = load_images(cancer_folder_path, label=1)  # 1 for cancer
non_cancer_images, non_cancer_labels = load_images(non_cancer_folder_path, label=0)  # 0 for non-cancer

# Combine datasets
X = np.concatenate((cancer_images, non_cancer_images), axis=0)
y = np.concatenate((cancer_labels, non_cancer_labels), axis=0)

# Normalize pixel values to [0, 1]
X = X / 255.0

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Plot training history
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Save the model
# model.save('cancer_classifier_model.h5')


Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\1\FILE0004.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\1\FILE0005.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\1\FILE0006.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\1\FILE0007.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\10\FILE0004.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\10\FILE0005.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\10\FILE0006.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\10\FILE0007.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\100\FILE0000.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\100\FILE0001.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\100\FI

KeyboardInterrupt: 

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Paths to folders containing images
cancer_folder_path = r"D:\yedek\lavdie.rada\Pictures\kancer"
non_cancer_folder_path = r"D:\yedek\lavdie.rada\Pictures\non cancer"

# Image dimensions
IMG_HEIGHT = 128
IMG_WIDTH = 128

# Load images and labels from subfolders
def load_images_from_subfolders(folder_path, label):
    images = []
    labels = []
    
    # Walk through all subfolders in the folder
    for root, dirs, files in os.walk(folder_path):  # This will walk through all subfolders
        for filename in files:
            file_path = os.path.join(root, filename)
            
            # Print the path for debugging
            print(f"Processing: {file_path} (is directory: {os.path.isdir(file_path)})")
            
            # Check if the file is an image (you can add more formats if needed)
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                try:
                    # Load and resize the image
                    img = tf.keras.preprocessing.image.load_img(file_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
                    img_array = tf.keras.preprocessing.image.img_to_array(img)
                    images.append(img_array)
                    labels.append(label)
                except Exception as e:
                    print(f"Failed to process {file_path}: {e}")
            else:
                print(f"Skipping non-image file: {filename}")
    return np.array(images), np.array(labels)

# Load cancer and non-cancer images
cancer_images, cancer_labels = load_images_from_subfolders(cancer_folder_path, label=1)  # 1 for cancer
non_cancer_images, non_cancer_labels = load_images_from_subfolders(non_cancer_folder_path, label=0)  # 0 for non-cancer

# Combine datasets
X = np.concatenate((cancer_images, non_cancer_images), axis=0)
y = np.concatenate((cancer_labels, non_cancer_labels), axis=0)

# Normalize pixel values to [0, 1]
X = X / 255.0

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

# Plot training history
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Save the model
# model.save('cancer_classifier_model.h5')


Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\1\FILE0004.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\1\FILE0005.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\1\FILE0006.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\1\FILE0007.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\10\FILE0004.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\10\FILE0005.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\10\FILE0006.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\10\FILE0007.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\100\FILE0000.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\100\FILE0001.jpg (is directory: False)
Processing: D:\yedek\lavdie.rada\Pictures\kancer\Cancer\100\FI